In [1]:
import sys
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
sys.path.append("..")
from mango.environments import frozen_lake
import utils

In [2]:
params = dict(map_name="RANDOM", p=None, shape=(4,4), seed=0)
env = frozen_lake.CustomFrozenLakeEnv(**params)
env = frozen_lake.wrappers.ReInitOnReset(env, **params)
env = frozen_lake.wrappers.TensorObservation(env, one_hot=True)
obs, info = env.reset()
print(f"Observation:\n {obs.shape, obs.dtype, obs.device}")
utils.render(env)

In [ ]:
from mango.mango import MangoEnv, Agent
from mango.policies.experiencereplay import ExperienceReplay
from mango.policies.dqnet import DQNetTrainer, DQNetPolicy
from mango.neuralnetworks.networks import ConvEncoder

net_kwargs = dict(hidden_channels=[32], hidden_features=[])
trainer_kwargs = dict(lr=1e-3, gamma=0.99, tau=1e-3)


net = ConvEncoder(in_channels=None, out_features=int(env.action_space.n), **net_kwargs).eval()
policy = DQNetPolicy(net, env.action_space)
trainer = DQNetTrainer(net, **trainer_kwargs)
replay = ExperienceReplay()
agent = Agent(MangoEnv(env), policy)

In [ ]:
import snakeviz
%load_ext snakeviz

In [ ]:
%%snakeviz -t
for episode in tqdm(range(100)):
    trans = agent.run_episode(randomness=0.5)
    for step in trans.steps:
        replay.push(step)

    if replay.can_sample():
        info = trainer.train(replay.sample())
        if episode % 10 == 0:
            print(f"Episode {episode}: {info.loss}")

  0%|          | 0/100 [00:00<?, ?it/s]

Episode 0: {'prob': 1}
Episode 10: {'prob': 1}
Episode 20: {'prob': 1}
Episode 30: TrainInfo(loss=tensor(0.1670, grad_fn=<SmoothL1LossBackward0>), td=tensor([[-0.0454],
        [-0.0156],
        [-0.2820],
        [-0.2614],
        [ 0.2418],
        [-0.8913],
        [-0.9726],
        [-1.0401],
        [-0.4414],
        [-0.0419],
        [-1.0755],
        [-0.3983],
        [-0.7243],
        [ 0.6296],
        [-0.5068],
        [-0.0419],
        [-0.3752],
        [-0.7431],
        [-0.3881],
        [-1.3517],
        [-1.1284],
        [ 0.3007],
        [-0.9306],
        [ 0.3031],
        [ 0.0115],
        [-0.3597],
        [ 0.5349],
        [-0.6784],
        [ 0.1279],
        [-0.0171],
        [-0.6813],
        [-0.4313],
        [-0.5227],
        [-0.6146],
        [-0.2512],
        [-0.0600],
        [-1.5105],
        [-0.3622],
        [-0.4748],
        [-0.1362],
        [ 0.0488],
        [-0.0142],
        [ 0.3734],
        [ 0.2223],
        [-1.07

In [ ]:
trans = agent.run_episode(randomness=0.0)
utils.render(env, trajectory=trans.trajectory)

NameError: name 'agent' is not defined